In [1]:
# Tạo model toán học
import gurobipy as gp

from gurobipy import GRB

# Tạo model
m = gp.Model("mip1")

Restricted license - for non-production use only - expires 2024-10-28


In [2]:
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

In [3]:
m

<gurobi.Model Continuous instance mip1: 0 constrs, 0 vars, No parameter changes>

In [4]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x3eaaf627
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 6 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [ ]:
import numpy as np

def column_generation(c, A, b):
    # Initialize the list of indices of basic variables
    basis = [1, 2, 3]
    
    # Extract the submatrices of A and c corresponding to the basic variables
    B = A[:, basis]
    c_B = c[basis]
    
    # Calculate the basic solution
    x_B = np.linalg.pinv(B) @ b
    
    # Calculate the dual variables
    y = np.linalg.pinv(B.T) @ c_B
    reduced_costs = c - A.T @ y
    
    # Check if the solution is optimal
    if np.all(reduced_costs >= 0):
        return x_B[basis], np.dot(x_B, c_B)
    
    # Find the index of the most negative reduced cost
    entering_var = np.argmin(reduced_costs)
    
    # Calculate the new column
    new_col = np.linalg.pinv(B) @ A[:, entering_var]
    
    # Add the new column to the basis
    basis.append(entering_var)
    
    # Recursively solve the subproblem
    x, obj_val = column_generation(c, A, b)
    
    return x, obj_val


# Define the problem data
c = np.array([1, 1, 1, 0, 0, 0])
A = np.array([
    [1, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1],
    [1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
])
b = np.array([3, 3, 2, 2, 2])

# Solve the problem using column generation
x, obj_val = column_generation(c, A, b)

# Print the solution
print("x =", x)
print("obj_val =", obj_val)
